In [ ]:
import pandas as pd
import sys
mod = sys.modules[__name__]
from sklearn import ensemble
import numpy as np

Data = pd.read_csv(r'D:\Desktop\SynologyDrive\하강우\bubble\data\bubble.csv')
R_csv = pd.read_csv(r'D:\Desktop\SynologyDrive\하강우\bubble\data\R.csv')

num_Fold = 3

mean = Data.mean()[-1]
std = Data.std()[-1]

def stz(Data):
    return (Data - Data.mean())/Data.std()

def norm(Data, min_norm=-0.9, max_norm=0.9):
    return (Data - Data.min())/(Data.max() - Data.min())*(max_norm - min_norm) + min_norm

stz_Data = stz(Data)
norm_Data = norm(Data)

All_Data = pd.DataFrame()
All_Data[Data.columns[0:3]] = norm_Data[Data.columns[0:3]]
All_Data[Data.columns[3]] = stz_Data[Data.columns[3]]
All_Data[Data.columns[4]] = norm_Data[Data.columns[4]]
All_Data[Data.columns[5]] = stz_Data[Data.columns[5]]
All_Data['R'] = R_csv[['R']].iloc[:len(Data)]

In [ ]:
case_list = [[0,5],[5,20],[20,35],[35,50],[50,65],[65,80],[80,95],[95,110],[110,125],[125,140],
             [140,155],[155,162],[162,169],[169,176],[176,183],[183,198],[198,213],[213,228],
             [228,243],[243,258],[258,273],[273,288],[288,302]]

for j in range(5):
    
    for i in range(len(case_list)):
        setattr(mod,'case{}_RF_Data_{}'.format(i+1,j+1),pd.read_csv(r'D:\Desktop\SynologyDrive\하강우\bubble\RF_case_modeling\RF_case{}_Data{}.csv'.format(i+1,j+1)))
for i in range(len(case_list)):
    
    setattr(mod,'case{}_RF_Data_model'.format(i+1), pd.DataFrame())
    getattr(mod,'case{}_RF_Data_model'.format(i+1))['model'] = [0 for i in range(case_list[i][1] - case_list[i][0])]
    
    setattr(mod,'case{}_RF_Data_y_data'.format(i+1), pd.DataFrame())
    getattr(mod,'case{}_RF_Data_y_data'.format(i+1))['y_data'] = [0 for i in range(case_list[i][1] - case_list[i][0])]
    
    for j in range(5):
        
        getattr(mod,'case{}_RF_Data_model'.format(i+1))['model{}'.format(j+1)] = getattr(mod,'case{}_RF_Data_{}'.format(i+1,j+1))['model'].iloc[case_list[i][0]:case_list[i][1]].reset_index(drop=True)
        
        getattr(mod,'case{}_RF_Data_y_data'.format(i+1))['y_data{}'.format(j+1)] = getattr(mod,'case{}_RF_Data_{}'.format(i+1,j+1))['y_data'].iloc[case_list[i][0]:case_list[i][1]].reset_index(drop=True)
        
    del getattr(mod,'case{}_RF_Data_model'.format(i+1))['model']
    del getattr(mod,'case{}_RF_Data_y_data'.format(i+1))['y_data']
    
    setattr(mod,'avg_case{}_RF_Data'.format(i+1), pd.DataFrame())
    getattr(mod,'avg_case{}_RF_Data'.format(i+1))['model']  = getattr(mod,'case{}_RF_Data_model'.format(i+1)).mean(axis=1)
    getattr(mod,'avg_case{}_RF_Data'.format(i+1))['y_data']  = getattr(mod,'case{}_RF_Data_y_data'.format(i+1)).mean(axis=1)
    
    setattr(mod,'std_case{}_RF_Data'.format(i+1), pd.DataFrame())
    getattr(mod,'std_case{}_RF_Data'.format(i+1))['model']  = getattr(mod,'case{}_RF_Data_model'.format(i+1)).std(axis=1)
    getattr(mod,'std_case{}_RF_Data'.format(i+1))['y_data']  = getattr(mod,'case{}_RF_Data_y_data'.format(i+1)).std(axis=1)
        

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

x = np.linspace(0,12)
y1 = x*1.3
y2 = x*0.7

plt.gca().set_aspect("equal")
for i in 4,5,7,8,9,10,11,12,13,14,15,20,21,22,23:
    plt.scatter(getattr(mod,'avg_case{}_RF_Data'.format(i))['model'],getattr(mod,'avg_case{}_RF_Data'.format(i))['y_data'],s=10,color='b')
plt.plot([0,12],[0,12],color='k')
plt.plot(x,y1,'--',color='k')
plt.plot(x,y2,'--',color='k')
plt.axis([0,12,0,12])
plt.grid(True)
plt.xlabel('$D_{sm}^{RF}$ (mm)')
plt.ylabel('$D_{sm}^{EXP}$ (mm)')
plt.title('Random Forest interpolation case avg scatter')
plt.savefig(r'D:\Desktop\SynologyDrive\하강우\bubble\wallcoresnu\RF_inter_scatter.png',dpi=300)

In [ ]:
def MSE(A,B):
    return ((A - B)**2).mean()

def ARE(A,T):
    return (abs((A-T)/T)*100).mean()

aa = []
bb = []
for i in 4,5,7,8,9,10,11,12,13,14,15,20,21,22,23:
    aa.append(MSE(getattr(mod,'avg_case{}_RF_Data'.format(i))['model'],getattr(mod,'avg_case{}_RF_Data'.format(i))['y_data']))
    bb.append(ARE(getattr(mod,'avg_case{}_RF_Data'.format(i))['model'],getattr(mod,'avg_case{}_RF_Data'.format(i))['y_data']))
    
print(np.sqrt(sum(aa)/len(aa)))
print(sum(bb)/len(bb))

In [ ]:
case_list = [[0,5],[5,20],[20,35],[35,50],[50,65],[65,80],[80,95],[95,110],[110,125],[125,140],
             [140,155],[155,162],[162,169],[169,176],[176,183],[183,198],[198,213],[213,228],
             [228,243],[243,258],[258,273],[273,288],[288,302]]
n_Avg = 5
n_Fold = 4

for j in range(n_Avg):
    for i in range(n_Fold):
        setattr(mod,'inter_Data_{}_{}'.format(i+1,j+1), pd.read_csv(r'D:\Desktop\SynologyDrive\하강우\bubble\K_Fold_Regular_ANN_1million_epoch\regular_ANN_{}_4_inter_Fold_million_iter_Data_1{}.csv'.format(i+1,j+1)))
        setattr(mod,'extra_Data_{}_{}'.format(i+1,j+1), pd.read_csv(r'D:\Desktop\SynologyDrive\하강우\bubble\K_Fold_Regular_ANN_1million_epoch\regular_ANN_{}_4_extra_Fold_million_iter_Data_1{}.csv'.format(i+1,j+1)))        

In [ ]:
# interpolation scatter
plt.gca().set_aspect("equal")
for j in range(n_Avg):
    for i in range(n_Fold):
        plt.scatter(getattr(mod,'inter_Data_{}_{}'.format(i+1,j+1))['model'],getattr(mod,'inter_Data_{}_{}'.format(i+1,j+1))['y_data'],s=10,color='b')
plt.plot([0,12],[0,12],color='k')
plt.plot(x,y1,'--',color='k')
plt.plot(x,y2,'--',color='k')
plt.axis([0,12,0,12])
plt.grid(True)
plt.xlabel('$D_{sm}^{ANN}$ (mm)')
plt.ylabel('$D_{sm}^{EXP}$ (mm)')
plt.title('1/4 case Regular ANN interpolation full scatter')
plt.savefig(r'D:\Desktop\SynologyDrive\하강우\bubble\wallcoresnu\Regular_ANN_4_Fold_inter_full_scatter.png',dpi=300)

In [ ]:
# interpolation scatter
plt.gca().set_aspect("equal")
for j in range(n_Avg):
    for i in range(n_Fold):
        plt.scatter(getattr(mod,'extra_Data_{}_{}'.format(i+1,j+1))['model'],getattr(mod,'extra_Data_{}_{}'.format(i+1,j+1))['y_data'],s=10,color='b')
plt.plot([0,12],[0,12],color='k')
plt.plot(x,y1,'--',color='k')
plt.plot(x,y2,'--',color='k')
plt.axis([0,12,0,12])
plt.grid(True)
plt.xlabel('$D_{sm}^{ANN}$ (mm)')
plt.ylabel('$D_{sm}^{EXP}$ (mm)')
plt.title('1/4 case Regular ANN extrapolation full scatter')
plt.savefig(r'D:\Desktop\SynologyDrive\하강우\bubble\wallcoresnu\Regular_ANN_4_Fold_extra_full_scatter.png',dpi=300)

In [ ]:
aa = []
bb = []
for j in range(n_Avg):
    for i in range(n_Fold):
        aa.append(MSE( getattr(mod,'inter_Data_{}_{}'.format(i+1,j+1))['model'],getattr(mod,'inter_Data_{}_{}'.format(i+1,j+1))['y_data'] ))
        bb.append(ARE( getattr(mod,'inter_Data_{}_{}'.format(i+1,j+1))['model'],getattr(mod,'inter_Data_{}_{}'.format(i+1,j+1))['y_data'] ))
        
print(np.sqrt(sum(aa)/len(aa)))
print(sum(bb)/len(bb))

In [ ]:
for j in range(n_Avg):
    for i in range(n_Fold):
        if i==0 and j==0:
            avg_inter_Data = pd.DataFrame()
            avg_inter_Data['model'] = [0 for i in range(len(B_Data))]
            avg_inter_Data['y_data'] = [0 for i in range(len(B_Data))]
            
            avg_inter_Data += inter_Data_1_1.set_index("index")
            avg_inter_Data = avg_inter_Data.replace(np.nan, 0)
            
        avg_inter_Data += getattr(mod,'inter_Data_{}_{}'.format(i+1,j+1)).set_index("index")

avg_inter_Data = avg_inter_Data/(n_Avg*(n_Fold ))

In [ ]:
plt.gca().set_aspect("equal")
plt.scatter(avg_inter_Data['model'],(B_Data['Dsm/R']*R_csv['R']).iloc[avg_inter_Data.index],s=10,color='b')
plt.plot([0,12],[0,12],color='k')
plt.plot(x,y1,'--',color='k')
plt.plot(x,y2,'--',color='k')
plt.axis([0,12,0,12])
plt.grid(True)
plt.xlabel('$D_{sm}^{ANN}$ (mm)')
plt.ylabel('$D_{sm}^{EXP}$ (mm)')
plt.title('1/4 case Regular ANN interpolation avg scatter')
plt.savefig(r'D:\Desktop\SynologyDrive\하강우\bubble\wallcoresnu\Regular_ANN_4_Fold_inter_avg_scatter.png',dpi=300)

In [ ]:
print(np.sqrt(MSE(avg_inter_Data['model'],avg_inter_Data['y_data'])))
print(ARE(avg_inter_Data['model'],avg_inter_Data['y_data']))

In [ ]:
avg_extra_Data = pd.DataFrame()
avg_extra_Data['model'] = [0 for i in range(len(B_Data))]
avg_extra_Data['y_data'] = [0 for i in range(len(B_Data))]

for j in range(n_Avg):
    for i in range(n_Fold):
        if j==0:
                       
            avg_extra_Data += getattr(mod,'extra_Data_{}_{}'.format(i+1,j+1)).set_index("index")
            avg_extra_Data = avg_extra_Data.replace(np.nan, 0)
            
        avg_extra_Data += getattr(mod,'extra_Data_{}_{}'.format(i+1,j+1)).set_index("index")

avg_extra_Data = avg_extra_Data/(n_Avg*(n_Fold ))

In [ ]:
plt.gca().set_aspect("equal")
plt.scatter(avg_extra_Data['model'],B_Data['Dsm/R']*R_csv['R'],s=10,color='b')
plt.plot([0,12],[0,12],color='k')
plt.plot(x,y1,'--',color='k')
plt.plot(x,y2,'--',color='k')
plt.axis([0,12,0,12])
plt.grid(True)
plt.xlabel('$D_{sm}^{ANN}$ (mm)')
plt.ylabel('$D_{sm}^{EXP}$ (mm)')
plt.title('1/4 case Regular ANN extrapolation avg scatter')
plt.savefig(r'D:\Desktop\SynologyDrive\하강우\bubble\wallcoresnu\Regular_ANN_4_Fold_extra_avg_scatter.png',dpi=300)

In [ ]:
avg_extra_Data_model = pd.DataFrame()
avg_extra_Data_model['model'] = [0 for i in range(len(B_Data))]

for j in range(n_Avg):
    for i in range(n_Fold):
        avg_extra_Data_model['model{}{}'.format(i+1,j+1)] = getattr(mod,'extra_Data_{}_{}'.format(i+1,j+1)).set_index('index')['model']
avg_extra_Data_model = avg_extra_Data_model.replace(np.nan, 0)
del avg_extra_Data_model['model']

avg_extra_Data = pd.DataFrame()
avg_extra_Data['model'] = (avg_extra_Data_model.mean(axis=1))*20/5

std_extra_Data = pd.DataFrame()
std_extra_Data['model'] = np.sqrt((avg_extra_Data_model.var(axis=1))*20/5)



In [ ]:
avg_extra_Data

In [ ]:
avg_inter_Data_model = pd.DataFrame()
avg_inter_Data_model['model'] = [0 for i in range(len(B_Data))]

for j in range(n_Avg):
    for i in range(n_Fold):
        avg_inter_Data_model['model{}{}'.format(i+1,j+1)] = getattr(mod,'inter_Data_{}_{}'.format(i+1,j+1)).set_index('index')['model']
avg_inter_Data_model = avg_inter_Data_model.replace(np.nan, 0)
del avg_inter_Data_model['model']

avg_inter_Data = pd.DataFrame()
avg_inter_Data['model'] = (avg_inter_Data_model.mean(axis=1))*20/20
avg_inter_Data = avg_inter_Data[avg_inter_Data['model'] != 0]

In [ ]:
avg_inter_Data_model.head()

In [ ]:
Data = pd.read_csv(r'D:\Desktop\190925\channel_pr_t\Prt_data\orig_channel_all_Re.csv')
for i in 180,400,590:
    setattr(mod,'Data_{}'.format(i), Data[Data['Re'] == i])
    setattr(mod,'Data_{}'.format(i), getattr(mod,'Data_{}'.format(i))[getattr(mod,'Data_{}'.format(i))['X'] == 0])
    setattr(mod,'Data_{}'.format(i), getattr(mod,'Data_{}'.format(i))[getattr(mod,'Data_{}'.format(i))['Y'] < 1])

In [ ]:
Data_180.columns

In [ ]:
for i in 180,400,590:
    
    plt.scatter(getattr(mod,'Data_{}'.format(i))['Y'],getattr(mod,'Data_{}'.format(i))['Nu_t_lsq_n'],label='Re : {}'.format(i))
    plt.legend()
    plt.axis([-.05,.85,-.001,.006])
    plt.xticks([0,.1,.2,.3,.4,.5,.6,.7,.8])
    plt.grid(True)

In [ ]:
for i in 180,400,590:
    
    plt.scatter(getattr(mod,'Data_{}'.format(i))['Y'],(-0.08)*(getattr(mod,'Data_{}'.format(i))['tke_n']**2)/getattr(mod,'Data_{}'.format(i))['tke_diss_n'],label='Re : {}'.format(i))
    plt.legend()
    plt.axis([-.05,.85,-.001,.006])
    plt.xticks([0,.1,.2,.3,.4,.5,.6,.7,.8])
    plt.grid(True)

In [ ]:
for i in 180,400,590:
    setattr(mod,'channel_Data_{}'.format(i), pd.DataFrame())
    getattr(mod,'channel_Data_{}'.format(i))['tke_n'] = getattr(mod,'Data_{}'.format(i))['tke_n']
    getattr(mod,'channel_Data_{}'.format(i))['tke_diss_n'] = getattr(mod,'Data_{}'.format(i))['tke_diss_n']
    getattr(mod,'channel_Data_{}'.format(i))['Nu_t_lsq_n'] = getattr(mod,'Data_{}'.format(i))['Nu_t_lsq_n']
    getattr(mod,'channel_Data_{}'.format(i))['Y'] = getattr(mod,'Data_{}'.format(i))['Y']
    getattr(mod,'channel_Data_{}'.format(i)).to_csv(r'D:\Desktop\190925\channel_pr_t\Prt_data\channel_Data_{}.csv'.format(i),index=False,header=True)

In [ ]:
import tensorflow as tf
from functools import partial
import sys
mod = sys.modules[__name__]
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

num_output = 1
num_layer = 4
num_neuron = 30
activation_function = 'tanh'
# initializer = tf.keras.initializers.RandomUniform(minval=-1, maxval=1)
initializer = tf.keras.initializers.he_normal()

for i in 180,400,590:
    setattr(mod,'Data_{}'.format(i), pd.read_csv(r'D:\Desktop\190925\channel_pr_t\Prt_data\channel_Data_{}.csv'.format(i)))
    
def stz(Data):
    return (Data - Data.mean())/Data.std()

def norm(Data, min_norm=1, max_norm=2):
    return (Data - Data.min())/(Data.max() - Data.min())*(max_norm - min_norm) + min_norm

stz_Data_180 = stz(Data_180)
# stz_Data_180 = norm(stz1_Data_180)
stz_Data_180.boxplot()

In [ ]:
train_X = stz_Data_180[stz_Data_180.columns[:-2]].sample(frac=0.7)
train_Y = stz_Data_180[stz_Data_180.columns[-2:-1]].iloc[train_X.index]

test_X = stz_Data_180[stz_Data_180.columns[:-2]].drop(train_X.index)
test_Y = stz_Data_180[stz_Data_180.columns[-2:-1]].iloc[test_X.index]

In [ ]:
from sklearn import ensemble
for i in range(5):
    RFmodel = ensemble.RandomForestRegressor(n_estimators=2000, min_samples_split=2, min_samples_leaf=1, bootstrap=False, oob_score=False, 
                                                            max_depth=None, max_features='auto', random_state=None, n_jobs=2)

    RFmodel.fit(train_X, train_Y)

    RF_180 = pd.DataFrame()
    RF_180['model'] = RFmodel.predict(test_X).flatten()
    RF_180 = RF_180*Data_180.std()[-2] + Data_180.mean()[-2]
    RF_180['y_data'] = Data_180['Nu_t_lsq_n'].loc[test_X.index]
    RF_180['Y'] = Data_180['Y'].loc[test_X.index]
    RF_180.to_csv(r'D:\Desktop\190925\channel_pr_t\Prt_data\RF_model_180_{}.csv'.format(i+1),header=True,index=False)
    
    RF_400 = pd.DataFrame()
    RF_400['model'] = RFmodel.predict(stz(Data_400[Data_400.columns[:-2]])).flatten()
    RF_400 = RF_400*Data_180.std()[-2] + Data_180.mean()[-2]
    RF_400.to_csv(r'D:\Desktop\190925\channel_pr_t\Prt_data\RF_model_400_{}.csv'.format(i+1),header=True,index=False)
    
    RF_590 = pd.DataFrame()
    RF_590['model'] = RFmodel.predict(stz(Data_590[Data_590.columns[:-2]])).flatten()
    RF_590 = RF_590*Data_180.std()[-2] + Data_180.mean()[-2]
    RF_590.to_csv(r'D:\Desktop\190925\channel_pr_t\Prt_data\RF_model_590_{}.csv'.format(i+1),header=True,index=False)

In [ ]:
import sys
mod = sys.modules[__name__]
import pandas as pd

for k in range(5):
    
    import tensorflow as tf
    from functools import partial

    num_output = 1
    num_layer = 10
    num_neuron = 20
    activation_function = 'elu'
    # initializer = tf.keras.initializers.RandomUniform(minval=-1, maxval=1)
    initializer = tf.keras.initializers.he_normal()

#     for i in 180,400,590:
#         setattr(mod,'Data_{}'.format(i), pd.read_csv(r'D:\Desktop\190925\channel_pr_t\Prt_data\channel_Data_{}.csv'.format(i)))

#     def stz(Data):
#         return (Data - Data.mean())/Data.std()

#     def norm(Data, min_norm=1, max_norm=2):
#         return (Data - Data.min())/(Data.max() - Data.min())*(max_norm - min_norm) + min_norm

#     stz_Data_180 = stz(Data_180)
#     stz_Data_180 = norm(stz_Data_180)
#     stz_Data_180.boxplot()
    
    l2_reg = tf.contrib.layers.l2_regularizer(scale=0.01)    
    dense_layer = partial(tf.layers.dense, activation=None, kernel_initializer=initializer, use_bias=True, bias_initializer=initializer, kernel_regularizer=l2_reg)

    X = tf.placeholder(tf.float32, shape=[None, 2])
    Y = tf.placeholder(tf.float32, shape=[None, 1])

    for i in range(num_layer):
        if i == 0:
            setattr(mod, 'hidden{}'.format(i+1),dense_layer(X,num_neuron))

        else:
            setattr(mod, 'hidden{}'.format(i+1),dense_layer(getattr(mod, 'hidden{}_act'.format(i)),num_neuron))

        setattr(mod,'hidden{}_act'.format(i+1),getattr(tf.nn,activation_function)(getattr(mod,'hidden{}'.format(i+1))))

    model = dense_layer(getattr(mod,'hidden{}'.format(num_layer)), num_output)

    mse_loss = tf.reduce_mean(tf.square(model - Y))
    kl_loss = tf.reduce_mean(tf.square(model/Y))
    
    loss_regularization = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    loss = tf.add_n([kl_loss] + loss_regularization)

    train_op = tf.train.AdamOptimizer(0.00001).minimize(loss)

    sess= tf.Session()
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())

    train_mse_profile = []
    test_mse_profile = []

    for epoch in range(50000):

#         batch_X = train_X.sample(n=2)
#         batch_Y = train_Y.loc[batch_X.index]
#         sess.run(train_op,feed_dict={X: batch_X, Y: batch_Y})
        sess.run(train_op, feed_dict={X: train_X, Y: train_Y})

        if (epoch+1) % 100 == 0:
            train_mse_profile.append(sess.run(mse_loss, feed_dict={X: train_X, Y: train_Y}))
            test_mse_profile.append(sess.run(mse_loss, feed_dict={X: test_X, Y: test_Y}))
            print('{} epochs mse train : {} , test : {}'.format(epoch+1,train_mse_profile[-1],test_mse_profile[-1]))


    SGD_180 = pd.DataFrame()
    SGD_180['model'] = sess.run(model,feed_dict={X: test_X}).flatten()
    SGD_180 = SGD_180*Data_180.std()[-2] + Data_180.mean()[-2]
    SGD_180['y_data'] = Data_180['Nu_t_lsq_n'].loc[test_X.index]
    SGD_180['Y'] = Data_180['Y'].loc[test_X.index]
    SGD_180.to_csv(r'D:\Desktop\190925\channel_pr_t\Prt_data\kl_L10_L2_SGD_model_180_{}.csv'.format(k+1),header=True,index=False)
    
    
    Data_400['tke_n'] = (Data_400['tke_n'] - Data_180['tke_n'].mean())/Data_180['tke_n'].std()
    Data_400['tke_diss_n'] = (Data_400['tke_diss_n'] - Data_180['tke_diss_n'].mean())/Data_180['tke_diss_n'].std()
    Data_400['Nu_t_lsq_n'] = (Data_400['Nu_t_lsq_n'] - Data_180['Nu_t_lsq_n'].mean())/Data_180['Nu_t_lsq_n'].std()

    ttt = pd.DataFrame()
    ttt['model'] = sess.run(model,feed_dict={X: Data_400[Data_400.columns[:-2]]}).flatten()
    ttt['y_data'] = Data_400[Data_400.columns[-2:-1]]
    ttt = ttt*Data_180.std()[-2] + Data_180.mean()[-2]
    ttt.to_csv(r'D:\Desktop\190925\channel_pr_t\Prt_data\kl_L10_L2_SGD_model_400_{}.csv'.format(k+1),header=True,index=False)
    
    Data_590['tke_n'] = (Data_590['tke_n'] - Data_180['tke_n'].mean())/Data_180['tke_n'].std()
    Data_590['tke_diss_n'] = (Data_590['tke_diss_n'] - Data_180['tke_diss_n'].mean())/Data_180['tke_diss_n'].std()
    Data_590['Nu_t_lsq_n'] = (Data_590['Nu_t_lsq_n'] - Data_180['Nu_t_lsq_n'].mean())/Data_180['Nu_t_lsq_n'].std()

    ttt = pd.DataFrame()
    ttt['model'] = sess.run(model,feed_dict={X: Data_590[Data_590.columns[:-2]]}).flatten()
    ttt['y_data'] = Data_590[Data_590.columns[-2:-1]]
    ttt = ttt*Data_180.std()[-2] + Data_180.mean()[-2]
    ttt.to_csv(r'D:\Desktop\190925\channel_pr_t\Prt_data\kl_L10_L2_SGD_model_590_{}.csv'.format(k+1),header=True,index=False)
    
    

In [ ]:
aa = [i*100 for i in range(len(train_mse_profile))]
plt.plot(aa,train_mse_profile,label='train')
plt.plot(aa,test_mse_profile,label='test')
plt.axis([0,50000,0,0.01])
plt.legend()
plt.xlabel('epoch')
plt.ylabel('Mean Squared Error')
plt.savefig(r'D:\Desktop\190925\channel_pr_t\Prt_data\batch_train_epoch_profile.png',dpi=300)

In [ ]:
plt.scatter(Data_180['Y'].loc[test_X.index],Data_180['Nu_t_lsq_n'].loc[test_X.index],s=10,label='DNS')
plt.scatter(Data_180['Y'].loc[test_X.index],RF_180['model'],s=10,label='Random Forest')
plt.axis([-.05,.85,-.001,.006])
plt.xticks([0,.1,.2,.3,.4,.5,.6,.7,.8])
plt.grid(True)
plt.legend()
plt.savefig(r'D:\Desktop\190925\channel_pr_t\Prt_data\RF_Re180_interpolation.png',dpi=300)

In [ ]:
for i in 180,400,590:
        setattr(mod,'Data_{}'.format(i), pd.read_csv(r'D:\Desktop\190925\channel_pr_t\Prt_data\channel_Data_{}.csv'.format(i)))

ttt = pd.DataFrame()
ttt['model'] = sess.run(model,feed_dict={X: Data_400[Data_400.columns[:-2]]}).flatten()
ttt['y_data'] = Data_400[Data_400.columns[-2:-1]]
ttt = ttt*Data_180.std()[-2] + Data_180.mean()[-2]

plt.scatter(Data_400['Y'],Data_400['Nu_t_lsq_n'],s=10,label='DNS')
plt.scatter(Data_400['Y'],ttt['model'],s=10,label='Random Forest')
plt.axis([-.05,.85,-.001,.006])
plt.xticks([0,.1,.2,.3,.4,.5,.6,.7,.8])
plt.grid(True)
plt.legend()
# plt.savefig(r'D:\Desktop\190925\channel_pr_t\Prt_data\RF_Re400_interpolation.png',dpi=300)

In [ ]:
tttt = pd.DataFrame()
tttt['model'] = sess.run(model,feed_dict={X: stz(Data_400[Data_400.columns[:-2]])}).flatten()
tttt = tttt*Data_180.std()[-2] + Data_180.mean()[-2]

plt.scatter(Data_590['Y'],Data_590['Nu_t_lsq_n'],s=10,label='DNS')
plt.scatter(Data_590['Y'],tttt['model'],s=10,label='Random Forest')
plt.axis([-.05,.85,-.001,.006])
plt.xticks([0,.1,.2,.3,.4,.5,.6,.7,.8])
plt.grid(True)
plt.legend()
# plt.savefig(r'D:\Desktop\190925\channel_pr_t\Prt_data\RF_Re590_interpolation.png',dpi=300)

In [ ]:
print(ARE(tt['model'],Data_180['Nu_t_lsq_n'].loc[test_X.index]))
print(ARE(ttt['model'],Data_400['Nu_t_lsq_n']))
print(ARE(tttt['model'],Data_590['Nu_t_lsq_n']))

In [ ]:
for j in 180,400,590:
    for i in range(5):
        setattr(mod,'L15_L2_SGD_{}_{}'.format(j,i+1), pd.read_csv(r'D:\Desktop\190925\channel_pr_t\Prt_data\L15_L2_SGD_model_{}_{}.csv'.format(j,i+1)))
              
for j in 180,400,590:
    setattr(mod,'avg_L15_L2_SGD_{}_model'.format(j), pd.DataFrame())
    getattr(mod,'avg_L15_L2_SGD_{}_model'.format(j))['model'] = [0 for i in range(len(getattr(mod,'L15_L2_SGD_{}_1'.format(j))))]
    for i in range(5):
        getattr(mod,'avg_L15_L2_SGD_{}_model'.format(j))['model{}'.format(i+1)] = getattr(mod,'L15_L2_SGD_{}_{}'.format(j,i+1))['model']
        
    del getattr(mod,'avg_L15_L2_SGD_{}_model'.format(j))['model']
    setattr(mod,'avg_L15_L2_SGD_{}'.format(j), getattr(mod,'avg_L15_L2_SGD_{}_model'.format(j)).mean(axis=1))
    setattr(mod,'std_L15_L2_SGD_{}'.format(j), getattr(mod,'avg_L15_L2_SGD_{}_model'.format(j)).std(axis=1))

In [ ]:
for j in 180,400,590:
    for i in range(5):
        setattr(mod,'L10_L2_Regular_{}_{}'.format(j,i+1), pd.read_csv(r'D:\Desktop\190925\channel_pr_t\Prt_data\L10_L2_Regular_model_{}_{}.csv'.format(j,i+1)))
              
for j in 180,400,590:
    setattr(mod,'avg_L10_L2_Regular_{}_model'.format(j), pd.DataFrame())
    getattr(mod,'avg_L10_L2_Regular_{}_model'.format(j))['model'] = [0 for i in range(len(getattr(mod,'L10_L2_Regular_{}_1'.format(j))))]
    for i in range(5):
        getattr(mod,'avg_L10_L2_Regular_{}_model'.format(j))['model{}'.format(i+1)] = getattr(mod,'L10_L2_Regular_{}_{}'.format(j,i+1))['model']
        
    del getattr(mod,'avg_L10_L2_Regular_{}_model'.format(j))['model']
    setattr(mod,'avg_L10_L2_Regular_{}'.format(j), getattr(mod,'avg_L10_L2_Regular_{}_model'.format(j)).mean(axis=1))
    setattr(mod,'std_L10_L2_Regular_{}'.format(j), getattr(mod,'avg_L10_L2_Regular_{}_model'.format(j)).std(axis=1))

In [ ]:
for j in 180,400,590:
    for i in range(5):
        setattr(mod,'L10_L2_SGD_{}_{}'.format(j,i+1), pd.read_csv(r'D:\Desktop\190925\channel_pr_t\Prt_data\L10_L2_SGD_model_{}_{}.csv'.format(j,i+1)))
              
for j in 180,400,590:
    setattr(mod,'avg_L10_L2_SGD_{}_model'.format(j), pd.DataFrame())
    getattr(mod,'avg_L10_L2_SGD_{}_model'.format(j))['model'] = [0 for i in range(len(getattr(mod,'L10_L2_SGD_{}_1'.format(j))))]
    for i in range(5):
        getattr(mod,'avg_L10_L2_SGD_{}_model'.format(j))['model{}'.format(i+1)] = getattr(mod,'L10_L2_SGD_{}_{}'.format(j,i+1))['model']
        
    del getattr(mod,'avg_L10_L2_SGD_{}_model'.format(j))['model']
    setattr(mod,'avg_L10_L2_SGD_{}'.format(j), getattr(mod,'avg_L10_L2_SGD_{}_model'.format(j)).mean(axis=1))
    setattr(mod,'std_L10_L2_SGD_{}'.format(j), getattr(mod,'avg_L10_L2_SGD_{}_model'.format(j)).std(axis=1))

In [ ]:
for j in 180,400,590:
    for i in range(5):
        setattr(mod,'L2_SGD_{}_{}'.format(j,i+1), pd.read_csv(r'D:\Desktop\190925\channel_pr_t\Prt_data\L2_SGD_model_{}_{}.csv'.format(j,i+1)))
              
for j in 180,400,590:
    setattr(mod,'avg_L2_SGD_{}_model'.format(j), pd.DataFrame())
    getattr(mod,'avg_L2_SGD_{}_model'.format(j))['model'] = [0 for i in range(len(getattr(mod,'L2_SGD_{}_1'.format(j))))]
    for i in range(5):
        getattr(mod,'avg_L2_SGD_{}_model'.format(j))['model{}'.format(i+1)] = getattr(mod,'L2_SGD_{}_{}'.format(j,i+1))['model']
        
    del getattr(mod,'avg_L2_SGD_{}_model'.format(j))['model']
    setattr(mod,'avg_L2_SGD_{}'.format(j), getattr(mod,'avg_L2_SGD_{}_model'.format(j)).mean(axis=1))
    setattr(mod,'std_L2_SGD_{}'.format(j), getattr(mod,'avg_L2_SGD_{}_model'.format(j)).std(axis=1))

In [ ]:
import pandas as pd
import sys
mod = sys.modules[__name__]
import matplotlib.pyplot as plt
%matplotlib inline

for j in 180,400,590:
    for i in range(20):
        setattr(mod,'SGD_{}_{}'.format(j,i+1), pd.read_csv(r'D:\Desktop\190925\channel_pr_t\Prt_data\SGD_model_{}_{}.csv'.format(j,i+1)))
              
for j in 180,400,590:
    setattr(mod,'avg_SGD_{}_model'.format(j), pd.DataFrame())
    getattr(mod,'avg_SGD_{}_model'.format(j))['model'] = [0 for i in range(len(getattr(mod,'SGD_{}_1'.format(j))))]
    for i in range(5):
        getattr(mod,'avg_SGD_{}_model'.format(j))['model{}'.format(i+1)] = getattr(mod,'SGD_{}_{}'.format(j,i+1))['model']
        
    del getattr(mod,'avg_SGD_{}_model'.format(j))['model']
    setattr(mod,'avg_SGD_{}'.format(j), getattr(mod,'avg_SGD_{}_model'.format(j)).mean(axis=1))
    setattr(mod,'std_SGD_{}'.format(j), getattr(mod,'avg_SGD_{}_model'.format(j)).std(axis=1))

In [ ]:
for j in 180,400,590:
    for i in range(5):
        setattr(mod,'RA_{}_{}'.format(j,i+1), pd.read_csv(r'D:\Desktop\190925\channel_pr_t\Prt_data\RA_model_{}_{}.csv'.format(j,i+1)))
              
for j in 180,400,590:
    setattr(mod,'avg_RA_{}_model'.format(j), pd.DataFrame())
    getattr(mod,'avg_RA_{}_model'.format(j))['model'] = [0 for i in range(len(getattr(mod,'RA_{}_1'.format(j))))]
    for i in range(5):
        getattr(mod,'avg_RA_{}_model'.format(j))['model{}'.format(i+1)] = getattr(mod,'RA_{}_{}'.format(j,i+1))['model']
        
    del getattr(mod,'avg_RA_{}_model'.format(j))['model']
    setattr(mod,'avg_RA_{}'.format(j), getattr(mod,'avg_RA_{}_model'.format(j)).mean(axis=1))
    setattr(mod,'std_RA_{}'.format(j), getattr(mod,'avg_RA_{}_model'.format(j)).std(axis=1))

In [ ]:
for j in 180,400,590:
    for i in range(5):
        setattr(mod,'RF_{}_{}'.format(j,i+1), pd.read_csv(r'D:\Desktop\190925\channel_pr_t\Prt_data\RF_model_{}_{}.csv'.format(j,i+1)))
              
for j in 180,400,590:
    setattr(mod,'avg_RF_{}_model'.format(j), pd.DataFrame())
    getattr(mod,'avg_RF_{}_model'.format(j))['model'] = [0 for i in range(len(getattr(mod,'RF_{}_1'.format(j))))]
    for i in range(5):
        getattr(mod,'avg_RF_{}_model'.format(j))['model{}'.format(i+1)] = getattr(mod,'RF_{}_{}'.format(j,i+1))['model']
        
    del getattr(mod,'avg_RF_{}_model'.format(j))['model']
    setattr(mod,'avg_RF_{}'.format(j), getattr(mod,'avg_RF_{}_model'.format(j)).mean(axis=1))
    setattr(mod,'std_RF_{}'.format(j), getattr(mod,'avg_RF_{}_model'.format(j)).std(axis=1))

In [ ]:
# Re 180 interpolation
plt.scatter(Data_180['Y'].loc[test_X.index]*180,Data_180['Nu_t_lsq_n'].loc[test_X.index],s=40,color='b',label='DNS (in 2019 S.Kang et al)')
plt.scatter(Data_180['Y'].loc[test_X.index]*180,(-0.11*((Data['tke_n']**2)/Data['tke_diss_n'])).loc[test_X.index],s=20,color='purple',label='k-$\epsilon$ model')
plt.errorbar(Data_180['Y'].loc[test_X.index], avg_L2_SGD_180,yerr=1.96*std_L2_SGD_180,fmt='rs',ms=4,linewidth=1,capsize=2,label='L8 L2 reg SGD ANN')
plt.errorbar(Data_180['Y'].loc[test_X.index], avg_L10_L2_SGD_180,yerr=1.96*std_L10_L2_SGD_180,fmt='ys',ms=4,linewidth=1,capsize=2,label='L10 L2 reg SGD ANN')
plt.errorbar(Data_180['Y'].loc[test_X.index], avg_L15_L2_SGD_180,yerr=1.96*std_L15_L2_SGD_180,fmt='gs',ms=4,linewidth=1,capsize=2,label='L15 L2 reg SGD ANN')
# plt.errorbar(Data_180['Y'].loc[test_X.index], avg_SGD_180,yerr=1.96*std_SGD_180,fmt='rs',ms=4,linewidth=1,capsize=2,label='SGD ANN')
# plt.errorbar(Data_180['Y'].loc[test_X.index]*180, avg_L10_L2_Regular_180,yerr=1.96*std_L10_L2_Regular_180,fmt='ys',ms=2,linewidth=1,capsize=2,label='L10 L2reg Regular ANN')
# plt.errorbar(Data_180['Y'].loc[test_X.index]*180, avg_RF_180,yerr=1.96*std_RF_180,fmt='gs',ms=3,linewidth=1,capsize=2,label='Random Forest')
plt.axis([-.05,0.9,-.001,.006])
# plt.xticks([0,.1,.2,.3,.4,.5,.6,.7,.8])
plt.grid(True)
plt.legend()
# plt.savefig(r'D:\Desktop\190925\channel_pr_t\Prt_data\y_plus_Re_180_profile.png',dpi=300)

In [ ]:
plt.gca().set_aspect("equal")
plt.scatter(Data_180['Nu_t_lsq_n'].loc[test_X.index],avg_RF_180,s=10,color='b')
plt.plot([0,0.005],[0,0.005],color='k')
plt.axis([0,0.005,0,0.005])
plt.grid(True)
plt.title('Random Forest interpolation')
plt.savefig(r'D:\Desktop\190925\channel_pr_t\Prt_data\RF_Re_180_scatter.png',dpi=300)

In [ ]:
print(np.sqrt(MSE(Data_180['Nu_t_lsq_n'].loc[test_X.index],avg_RF_180)))
print(ARE(Data_180['Nu_t_lsq_n'].loc[test_X.index],avg_RF_180))

In [ ]:
# Re 400 extrapolation
X = Data_400['Y'].sample(n=50)*400
plt.scatter(X,L2_SGD_400_1['y_data'].loc[X.index],s=20,color='b',label='DNS (in 2019 S.Kang et al)')
# plt.errorbar(X, avg_L15_L2_SGD_400.loc[X.index],yerr=1.96*std_L15_L2_SGD_400.loc[X.index],fmt='rs',ms=3,linewidth=1,capsize=2,label='L15 L2reg SGD ANN')
# plt.errorbar(X, avg_L10_L2_SGD_400.loc[X.index],yerr=1.96*std_L10_L2_SGD_400.loc[X.index],fmt='rs',ms=3,linewidth=1,capsize=2,label='L10 L2reg SGD ANN')
plt.scatter(X,(-0.09*((Data1['tke_n']**2)/Data1['tke_diss_n'])).loc[X.index],s=20,color='purple',label='k-$\epsilon$ model')
plt.errorbar(X, avg_L10_L2_Regular_400.loc[X.index],yerr=1.96*std_L10_L2_Regular_400.loc[X.index],fmt='ys',ms=3,linewidth=1,capsize=2,label='L10 L2 Regular ANN')
# plt.errorbar(X, avg_L2_SGD_400.loc[X.index],yerr=1.96*std_L2_SGD_400.loc[X.index],fmt='rs',ms=2,linewidth=1,capsize=2,label='L8 L2reg SGD ANN')
# plt.errorbar(X, avg_SGD_400.loc[X.index],yerr=1.96*std_SGD_400.loc[X.index],fmt='ys',ms=2,linewidth=1,capsize=2,label='L8 SGD ANN')
# plt.errorbar(X, avg_RA_400.loc[X.index],yerr=1.96*std_RA_400.loc[X.index],fmt='ys',ms=3,linewidth=1,capsize=2,label='Regular ANN')
plt.errorbar(X, avg_RF_400.loc[X.index],yerr=1.96*std_RF_400.loc[X.index],fmt='gs',ms=3,linewidth=1,capsize=2,label='Random Forest')
plt.axis([-.05,320,-.001,.006])
# plt.xticks([0,.1,.2,.3,.4,.5,.6,.7,.8])
plt.grid(True)
plt.legend()
plt.title('Re_tau 400 extrapolation case')
# plt.savefig(r'D:\Desktop\190925\channel_pr_t\Prt_data\y_plus_Re_400_profile.png',dpi=300)

In [ ]:
plt.gca().set_aspect("equal")
plt.scatter(SGD_400_1['y_data'].loc[X.index],avg_SGD_400.loc[X.index],s=10,color='b')
plt.plot([0,0.005],[0,0.005],color='k')
plt.axis([0,0.005,0,0.005])
plt.grid(True)
plt.title('400 SGD ANN extrapolation')
plt.savefig(r'D:\Desktop\190925\channel_pr_t\Prt_data\SGD_Re_400_scatter.png',dpi=300)

In [ ]:
# Re 590 extrapolation
# Re 400 extrapolation
X = Data_590['Y'].sample(n=50)*590
plt.scatter(X,L2_SGD_590_1['y_data'].loc[X.index],s=20,color='b',label='DNS (in 2019 S.Kang et al)')
plt.scatter(X,(-0.09*((Data1['tke_n']**2)/Data1['tke_diss_n'])).loc[X.index],s=20,color='purple',label='k-$\epsilon$ model')
# plt.errorbar(X, avg_L15_L2_SGD_590.loc[X.index],yerr=1.96*std_L15_L2_SGD_590.loc[X.index],fmt='rs',ms=2,linewidth=1,capsize=2,label='L15 L2reg SGD ANN')
# plt.errorbar(X, avg_L10_L2_SGD_590.loc[X.index],yerr=1.96*std_L10_L2_SGD_590.loc[X.index],fmt='rs',ms=3,linewidth=2,capsize=2,label='L10 L2reg SGD ANN')
plt.errorbar(X, avg_L10_L2_Regular_590.loc[X.index],yerr=1.96*std_L10_L2_Regular_590.loc[X.index],fmt='ys',ms=3,linewidth=1,capsize=2,label='L10 L2reg Regular ANN')
# plt.errorbar(X, avg_L2_SGD_590.loc[X.index],yerr=1.96*std_L2_SGD_590.loc[X.index],fmt='rs',ms=2,linewidth=0.5,capsize=2,label='L8 L2reg SGD ANN')
# plt.scatter(X,SGD_590_1['y_data'].loc[X.index],s=20,color='b',label='DNS (in 2019 S.Kang et al)')
# plt.scatter(X,(-0.117*((Data2['tke_n']**2)/Data2['tke_diss_n'])).loc[X.index],s=20,color='purple',label='k-$\epsilon$ model')
# plt.errorbar(X, avg_SGD_590.loc[X.index],yerr=1.96*std_SGD_590.loc[X.index],fmt='ys',ms=2,linewidth=1,capsize=2,label='L8 SGD ANN')
# plt.errorbar(X, avg_RA_590.loc[X.index],yerr=1.96*std_RA_590.loc[X.index],fmt='gs',ms=3,linewidth=1,capsize=2,label='Regular ANN')
plt.errorbar(X, avg_RF_590.loc[X.index],yerr=1.96*std_RF_590.loc[X.index],fmt='gs',ms=2,linewidth=1,capsize=2,label='Random Forest')
plt.axis([-.05,500,-.001,.006])
# plt.xticks([0,.1,.2,.3,.4,.5,.6,.7,.8])
plt.grid(True)
plt.legend()
plt.title('Re_tau 590 extrapolation case')
# plt.savefig(r'D:\Desktop\190925\channel_pr_t\Prt_data\y_plus_Re_590_profile.png',dpi=300)

In [ ]:
plt.gca().set_aspect("equal")
plt.scatter(SGD_590_1['y_data'].loc[X.index],avg_RF_590.loc[X.index],s=10,color='b')
plt.plot([0,0.005],[0,0.005],color='k')
plt.axis([0,0.005,0,0.005])
plt.grid(True)
plt.title('590 Random Forest extrapolation')
plt.savefig(r'D:\Desktop\190925\channel_pr_t\Prt_data\RF_Re_590_scatter.png',dpi=300)

In [ ]:
Data_400['tke_n'] = (Data_400['tke_n'] - Data_180['tke_n'].mean())/Data_180['tke_n'].std()
Data_400['tke_diss_n'] = (Data_400['tke_diss_n'] - Data_180['tke_diss_n'].mean())/Data_180['tke_diss_n'].std()
Data_400['Nu_t_lsq_n'] = (Data_400['Nu_t_lsq_n'] - Data_180['Nu_t_lsq_n'].mean())/Data_180['Nu_t_lsq_n'].std()

ttt = pd.DataFrame()
ttt['model'] = sess.run(model,feed_dict={X: Data_400[Data_400.columns[:-2]]}).flatten()
ttt['y_data'] = Data_400[Data_400.columns[-2:-1]]
# tttt['model'] = RFmodel.predict(stz(Data_590[Data_590.columns[:-2]]))
ttt = ttt*Data_180.std()[-2] + Data_180.mean()[-2]

plt.scatter(Data_400['Y'],Data_400['Nu_t_lsq_n'],s=10,label='DNS')
plt.scatter(Data_400['Y'],ttt['model'],s=10,label='Random Forest')
# plt.axis([-.05,.85,-.001,.006])
plt.xticks([0,.1,.2,.3,.4,.5,.6,.7,.8])
plt.grid(True)
plt.legend()
plt.savefig(r'D:\Desktop\190925\channel_pr_t\Prt_data\RF_Re400_interpolation.png',dpi=300)

In [ ]:
Data_400.boxplot()

In [ ]:
Data_590['tke_n'] = (Data_590['tke_n'] - Data_180['tke_n'].mean())/Data_180['tke_n'].std()
Data_590['tke_diss_n'] = (Data_590['tke_diss_n'] - Data_180['tke_diss_n'].mean())/Data_180['tke_diss_n'].std()
Data_590['Nu_t_lsq_n'] = (Data_590['Nu_t_lsq_n'] - Data_180['Nu_t_lsq_n'].mean())/Data_180['Nu_t_lsq_n'].std()

Data_590.boxplot()

In [ ]:
ttt = pd.DataFrame()
ttt['model'] = sess.run(model,feed_dict={X: Data_590[Data_590.columns[:-2]]}).flatten()
ttt['y_data'] = Data_590[Data_590.columns[-2:-1]]
# tttt['model'] = RFmodel.predict(stz(Data_590[Data_590.columns[:-2]]))
ttt = ttt*Data_180.std()[-2] + Data_180.mean()[-2]

plt.scatter(Data_590['Y'],ttt['y_data'],s=10,label='DNS')
plt.scatter(Data_590['Y'],ttt['model'],s=10,label='SGD ANN')
plt.axis([-.05,.85,-.001,.006])
plt.xticks([0,.1,.2,.3,.4,.5,.6,.7,.8])
plt.grid(True)
plt.legend()
plt.savefig(r'D:\Desktop\190925\channel_pr_t\Prt_data\RF_Re590_interpolation.png',dpi=300)

In [ ]:
Data_400.index

In [ ]:
Data = pd.read_csv(r'D:\Desktop\190925\channel_pr_t\Prt_data\channel_Data_180.csv')
Data1 = pd.read_csv(r'D:\Desktop\190925\channel_pr_t\Prt_data\channel_Data_400.csv')
Data2 = pd.read_csv(r'D:\Desktop\190925\channel_pr_t\Prt_data\channel_Data_590.csv')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline